In [11]:
import pandas as pd
import os
import openpyxl
import numpy as np
import re
from datetime import datetime

In [15]:
class FinancialReport():
    def __init__(self,file):
        self.file = file
        self.readFile()
        self.findBalanceSheet()
    def readFile(self):
        path = os.path.join("financial statements",self.file)
        self.statements = pd.read_excel(path,sheet_name=None)
    def getSheetNames(self):
        return self.statements.keys()
    def findBalanceSheet(self):
        balanceSheetName = [name for name in self.getSheetNames() if "BALANCE" in name][0]
        self.balance = self.statements[balanceSheetName]
        self.balance.columns = ["label"]+list(self.balance.columns[1:])
    def getBalanceSheet(self):
        return self.balance
    

In [39]:
class Balance():
    def __init__(self):
        self.finalBalance = pd.DataFrame(columns=["title","title_order","label"])
        self.setFinancialStatements()
        self.addDataToBalanceSheet()
        self.rearrangeFinalBalanceSheet()
        self.setQuarters()
        
    def getFiles(self):
        files = os.listdir("financial statements")
        files.remove(".DS_Store")
        files = [file for file in files if file[0] != "~"]
        return files

    def setFinancialStatements(self):
        self.files = self.getFiles()
        self.FinancialReports = [FinancialReport(file) for file in self.files]
        
    def addDataToBalanceSheet(self):
        
        for FinancialStatements in self.FinancialReports:
            balance = FinancialStatements.getBalanceSheet().copy()
            
            cols = ["label"]+[col for col in balance.columns[1:] if col not in self.finalBalance.columns] ## remove columns of data already in main balance sheet
            balance = balance[cols]
            
            balance = self.parseStatement(balance)
            
            self.finalBalance = pd.merge(self.finalBalance, balance, how="outer",on=["label","title"],suffixes=('', '_x'))
            
            self.finalBalance.title_order = [self.merge_number(self.finalBalance.title_order[i],self.finalBalance.title_order_x[i]) for i in range(len(self.finalBalance))] #merge the number for the titles. use greatest num bc bigger num means there is section before it
            self.finalBalance = self.finalBalance.drop(["title_order_x"],axis=1)

    def rearrangeFinalBalanceSheet(self):
        self.finalBalance = self.finalBalance.replace(" ",np.nan)

        # put totals at the bottom of each section
        totals = self.finalBalance.loc[self.finalBalance.label.str.startswith("total")].reset_index(drop=True) 
        self.finalBalance = self.finalBalance.loc[~self.finalBalance.label.str.startswith("total")].reset_index(drop=True)
        self.finalBalance = pd.concat([self.finalBalance,totals]) 
        self.finalBalance = self.finalBalance.reset_index(drop=True).reset_index()
        self.finalBalance = self.finalBalance.sort_values(["title_order","index"]) # order by title and then location
        self.finalBalance = self.finalBalance.drop(["index","title_order"],axis=1)

    def parseStatement(self,sheet):
        num = 0
        index = []
        title = False
        for label in sheet.label:
            if ":" in label:
                title = re.sub("\([\w\W]+\)","",label[:-1]).replace("  "," ")
                num += 1
                continue
            if title:
                new_label = label.lower()
                new_label = new_label.replace("gain","loss")
                new_label = re.sub("\([\w\W]+\)","",new_label).replace("  "," ").strip()
                new_label = re.sub(" \$|\s*\d+\,*", "", new_label) 
                new_label = self.clean_up_string(new_label)
                
                print([title,num,label,new_label])
                index.append([title,num,label,new_label])
            
            if "total" == new_label[:5]:
                title = np.nan
                num += 1

        sheet = sheet.loc[sheet.label.isin(np.array(index)[:,2])].copy()
        sheet.loc[:,["title","title_order","label"]] = list(np.array(index)[:,[0,1,3]])
        sheet = sheet.set_index(["title","title_order","label"]).reset_index()
        return sheet
    
    def clean_up_string(self,string):
        string = string.strip()
        space_positions = [x.start() for x in re.finditer('\ ',  string)]
        for i in reversed(space_positions):
            if string[i+1:] in ["of", "and"]:
                string = string[:i]
        return string
    
    def merge_number(self,a,b):
        if pd.isnull(a):
            return b
        if pd.isnull(b):
            return a
        return max(a,b)
    def setQuarters(self):
        dates = list(self.finalBalance.columns[2:])
        dates.sort(key=lambda date: datetime.strptime(date.replace(".",""), '%b %d, %Y'))
        self.finalBalance = self.finalBalance[["title","label"]+dates]
        self.finalBalance.columns = ["title","label"]+[self.parseQuarter(date) for date in self.finalBalance.columns[2:]]
    def parseQuarter(self,date):
        mo = date.split(" ")[0].strip(".")
        year = date.split(", ")[-1]
        if mo in ["May"]:
            return "Q1 "+str(int(year)+1)
        
        if mo in ["Jul","Aug"]:
            return "Q2 "+str(int(year)+1)
        
        if mo in ["Oct","Nov"]:
            return "Q3 "+str(int(year)+1)
        
        if mo in ["Jan","Feb"]:
            return "Q4 "+str(int(year))
        
        

In [40]:
balance = Balance()
balance.finalBalance

['Current assets', 1, 'Cash and cash equivalents', 'cash and cash equivalents']
['Current assets', 1, 'Short-term investments', 'short-term investments']
['Current assets', 1, 'Merchandise inventory', 'merchandise inventory']
['Current assets', 1, 'Other current assets', 'other current assets']
['Current assets', 1, 'Total current assets', 'total current assets']
[nan, 2, 'Property and equipment, net of accumulated depreciation', 'property and equipment, net of accumulated depreciation']
[nan, 2, 'Operating Lease, Right-of-Use Asset', 'operating lease, right-of-use asset']
[nan, 2, 'Other long-term assets', 'other long-term assets']
[nan, 2, 'Total assets', 'total assets']
['Current liabilities', 4, 'Accounts payable', 'accounts payable']
['Current liabilities', 4, 'Accrued expenses and other current liabilities', 'accrued expenses and other current liabilities']
['Current liabilities', 4, 'Operating Lease, Liability, Current', 'operating lease, liability, current']
['Current liabiliti

,title,label,Q4 2018,Q1 2019,Q2 2019,Q3 2019,Q4 2019,Q1 2020,Q2 2020,Q3 2020,Q4 2020,Q1 2021,Q2 2021,Q3 2021,Q4 2021,Q1 2022,Q2 2022,Q3 2022,Q4 2022
0,Current assets,cash and cash equivalents,1783.0,1210.0,1322.0,958.0,1081.0,941.0,1177.0,788.0,1364.0,1028.0,2188.0,2471.0,1988.0,2066.0,2375.0,801.0,877.00
1,Current assets,short-term investments,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,178.0,410.0,NaN,NaN,275.0,0.00
2,Current assets,merchandise inventory,1997.0,2035.0,2202.0,2668.0,2131.0,2242.0,2326.0,2720.0,2156.0,2217.0,2242.0,2747.0,2451.0,2370.0,2281.0,2721.0,3018.00
3,Current assets,other current assets,788.0,778.0,780.0,792.0,751.0,757.0,770.0,770.0,706.0,920.0,882.0,966.0,1159.0,1091.0,1201.0,1410.0,1270.00
23,Current assets,"available-for-sale securities, current",NaN,164.0,286.0,296.0,288.0,272.0,294.0,294.0,290.0,51.0,25.0,NaN,NaN,475.0,337.0,NaN,NaN
38,Current assets,"cash and cash equivalents, at carrying value",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,Current assets,"other assets, current",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,Current assets,total current assets,4568.0,4187.0,4590.0,4714.0,4251.0,4212.0,4567.0,4572.0,4516.0,4216.0,5337.0,6362.0,6008.0,6002.0,6194.0,5207.0,5165.00
19,nan,"property and equipment, accumulated depreciation",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5999.0,5839.0,5886.0,5933.0,5891.0,5608.0,5616.0,5603.0,5486.0,NaN
20,nan,"common stock, shares issued",NaN,NaN,NaN,NaN,NaN,NaN,NaN,373000.0,371000.0,373000.0,374000.0,374000.0,374000.0,377000.0,376000.0,374000.0,371000.00


In [24]:
balance.finalBalance.to_csv("finalBalance2.csv")

In [79]:
'Oct. 30, 2021'.split(", ")[-1]

'2021'